In [1]:
import deeplake
from matplotlib import pyplot as plt
import torch
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.metrics import accuracy, specificity
from Network.Unet import Custom2DUnet
from torch.nn.functional import one_hot
import numpy as np
import pandas as pd
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib import colors as mcolors
skip = False
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# CUDA_VISIBLE_DEVICES=3 python3 main.py -v FedAvgDrive --data-dir None  -cd None -e 4 -r 30 -bs 1 -m "fedavg" -t "drive"

/home/taehwan/FedRef/.venv/lib/python3.12/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.2.14) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
/home/taehwan/FedRef/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = deeplake.load('hub://activeloop/drive-test')
dataset["masks"].numpy()

hub://activeloop/drive-test loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/drive-test


array([[[[ True, False],
         [ True, False],
         [ True, False],
         ...,
         [ True, False],
         [ True, False],
         [ True, False]],

        [[ True, False],
         [ True, False],
         [ True, False],
         ...,
         [ True, False],
         [ True, False],
         [ True, False]],

        [[ True, False],
         [ True, False],
         [ True, False],
         ...,
         [ True, False],
         [ True, False],
         [ True, False]],

        ...,

        [[ True, False],
         [ True, False],
         [ True, False],
         ...,
         [ True, False],
         [ True, False],
         [ True, False]],

        [[ True, False],
         [ True, False],
         [ True, False],
         ...,
         [ True, False],
         [ True, False],
         [ True, False]],

        [[ True, False],
         [ True, False],
         [ True, False],
         ...,
         [ True, False],
         [ True, False],
         [ True, 

In [ ]:
def ploting(fedavg, fedprox, fedref, title="MNIST Classification", zoom_min = 20, zoom_max=25, inset=True, metric="loss", minimum_y=0):
    # 예시 데이터
    tableau_colors = list(mcolors.TABLEAU_COLORS.values())
    
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(fedavg[metric], marker=".",linestyle="--",label="fedavg", color=tableau_colors[0], alpha=1)
    ax.plot(fedprox[metric], marker=".", linestyle="--" ,label="fedprox(mu=0.5)", color=tableau_colors[1], alpha=0.7)
    for i,l in enumerate(fedref):
        ax.plot(l[metric], marker=".", label=l["label"], color=tableau_colors[i+2], linestyle="--")

    # 메인 축 설정
    ax.set_title(title)
    ax.set_xlabel("Round")
    ax.set_ylabel(metric)
    ax.legend()
    # 🔍 확대 구간 설정 (마지막 x값 근처, 예: x > 90)
    x_zoom_min = zoom_min
    x_zoom_max = zoom_max

    # inset 축 생성
    if inset:
        axins = inset_axes(ax, width="30%", height="40%", loc='lower right')
        axins.plot(fedavg[metric], color =tableau_colors[0], label = "fedavg", marker=".", alpha=1, linestyle="--")
        axins.plot(fedprox[metric], color =tableau_colors[1], label = "fedprox", linestyle="--", marker= ".", alpha=0.7)
        for i,l in enumerate(fedref):
            axins.plot(l[metric], label=l["label"], color=tableau_colors[i+2], linestyle="--", marker=".")

        alp = fedavg[metric][zoom_min:zoom_max+1].to_list() + fedprox[metric][zoom_min:zoom_max+1].to_list()
        for l in fedref:
            alp = alp + l[metric][zoom_min:zoom_max+1].to_list()

        axins.set_xlim(x_zoom_min, x_zoom_max)
        #확대된 y축 범위도 지정하면 보기 쉬움
        #y_zoom = y[(x >= x_zoom_min) & (x <= x_zoom_max)]
        axins.set_ylim(bottom=min(alp), top = max(alp))
        
        # 확대된 영역 표시선 연결
        ax.indicate_inset_zoom(axins)
    ax.legend(fontsize=11)

    minimum = [fedavg[metric].min(), fedprox[metric].min()] + [ s[metric].min() for s in fedref]
    minimum = min(minimum)
    ax.set_ylim(bottom=minimum-minimum_y)
    fig.show()
    

In [ ]:
diceLoss = smp.losses.DiceLoss(
   mode="binary",          # For multi-class segmentation
   classes=None,               # Compute the loss for all classes
   log_loss=False,             # Do not use log version of Dice loss
   from_logits=True,           # Model outputs are raw logits
   smooth=1e-5,                # A small smoothing factor for stability
   ignore_index=None,          # Don't ignore any classes
   eps=1e-7                    # Epsilon for numerical stability
)
class Hausdolf95(torch.nn.Module):
    def __init__(self):
        super(Hausdolf95, self).__init__()
        self.distancef = torch.nn.PairwiseDistance()
    def distance(self, x, y):
        return self.distancef(x, y)
        
    def forward(self, x, y):
        maximum_d=torch.max(self.distance(x, y))
        return maximum_d
hausdolf95 = Hausdolf95()
jaccardf = smp.losses.JaccardLoss(
   mode="binary",          # For multi-class segmentation
   classes=None,               # Compute the loss for all classes
   log_loss=False,             # Do not use log version of Dice loss
   from_logits=True,           # Model outputs are raw logits
   smooth=1e-5,                # A small smoothing factor for stability          
   eps=1e-7
)
def evaluation(net, loader, diceLoss, HF95, jaccard,DEVICE):
    net.eval()
    net.to(DEVICE)
    dicef= diceLoss.to(DEVICE)
    hf95f = HF95.to(DEVICE)
    jaccardf = jaccard.to(DEVICE)
    dice = 0
    hf95 = 0
    jacc = 0
    acc = 0
    spec = 0
    length = len(loader)
    for sample in loader:
        X= torch.stack([torch.Tensor(s["rgb_images"].numpy()).permute(-1,0,1) for s in sample], 0)
        Y= torch.stack([torch.where(torch.from_numpy(s['masks'].numpy()).squeeze()[...,0], 0.0, 1.0).type(torch.int64) for s in sample], 0)
        out = net(X.type(torch.float32).to(DEVICE))
        jacc += (1-jaccardf(out.squeeze(), Y.squeeze().type(torch.int64).to(DEVICE))).item()
        dice += (1-dicef(out.squeeze(), Y.squeeze().type(torch.int64).to(DEVICE))).item()
        hf95 += hf95f(out.squeeze(), Y.type(torch.float32).to(DEVICE)).item()
        out = (out>0.15).type(torch.int32).to(DEVICE)
        acc += accuracy(out.squeeze()*Y.to(DEVICE), (1-out).squeeze()*Y.to(DEVICE),out.squeeze()*(1-Y).to(DEVICE),(1-out).squeeze()*(1-Y).to(DEVICE)).mean().item()
        spec += specificity(out.squeeze()*Y.to(DEVICE), (1-out).squeeze()*Y.to(DEVICE), out.squeeze()*(1-Y).to(DEVICE), (1-out).squeeze()*(1-Y).to(DEVICE)).mean().item()
    return {"dice": dice/length, "hf95":hf95/length, 'jaccard': jacc/length, 'accuracy': acc/length, "specificity": spec/length}

In [ ]:
fedavg = pd.read_csv("Result/fedavg/fedavg_drive.csv")
fedprox = pd.read_csv("Result/fedprox/fedprox_drive.csv")
fedopt = pd.read_csv("Result/fedopt/fedopt_drive.csv")
fedref1 = pd.read_csv("Result/fedref1/fedref_drive_lda0.1_p3.csv")
ploting(fedavg, fedprox, [{"loss": fedopt["loss"], "label": "fedopt"}, {"loss": fedref1["loss"], "label": "fedref(lda: 0.01, prime: 3)"}], title="Drive Segmentation", zoom_min=20, zoom_max=25, inset=False,minimum_y=0.01)
ploting(fedavg, fedprox, [{"mDice": fedopt["mDice"], "label": "fedopt"}, {"mDice": fedref1["mDice"], "label": "fedref(lda: 0.01, prime: 3)"}], title="Drive Segmentation", zoom_min=20, zoom_max=25, inset=False, metric="mDice")
ploting(fedavg, fedprox, [{"mHF95": fedopt["mHF95"], "label": "fedopt"}, {"mHF95": fedref1["mHF95"], "label": "fedref(lda: 0.01, prime: 3)"}], title="Drive Segmentation", zoom_min=20, zoom_max=25, inset=False, metric="mHF95")

NameError: name 'pd' is not defined